In [1]:
import pandas as pd
import xgboost as xgb
import sys,random
import pickle
import os

In [2]:
if not os.path.exists('featurescore'):
    os.mkdir('featurescore')
if not os.path.exists('model'):
    os.mkdir('model')
if not os.path.exists('preds'):
    os.mkdir('preds')

In [3]:
%%time
#use rank_d  and rank_nd feature
#load data
train_x_d = pd.read_csv('../../preprocess_data/train_x_float_rank_d.csv')
train_x_nd = pd.read_csv('../../preprocess_data/train_x_float_rank_nd.csv')
train_x = pd.merge(train_x_d,train_x_nd,on='id')
train_y = pd.read_csv('../../preprocess_data/train_y_33465.csv')
train_x = train_x.drop(['id'],axis=1)
dtrain = xgb.DMatrix(train_x, label=train_y)

valid_d = pd.read_csv('../../preprocess_data/valid_float_rank_d.csv')
valid_nd = pd.read_csv('../../preprocess_data/valid_float_rank_nd.csv')
valid = pd.merge(valid_d,valid_nd,on='id')
valid_id = valid.id
valid = valid.drop("id",axis=1)
dvalid = xgb.DMatrix(valid)

CPU times: user 34.2 s, sys: 10.9 s, total: 45.1 s
Wall time: 45.9 s


In [4]:
def pipeline(dtrain,dtest,test_id,iteration,random_seed,gamma,max_depth,lambd,subsample,colsample_bytree,min_child_weight):
    if max_depth==6:
        num_boost_round = 750
    elif max_depth==7:
        num_boost_round = 600
    elif max_depth==8:
        num_boost_round = 450
    
    params={
    'booster':'gbtree',
    'objective': 'rank:pairwise',
#     'scale_pos_weight': float(len(y)-sum(y))/float(sum(y)),
    'eval_metric': 'auc',
    'gamma':gamma,
    'max_depth':max_depth,
    'lambda':lambd,
    'subsample':subsample,
    'colsample_bytree':colsample_bytree,
    'min_child_weight':min_child_weight, 
    'eta': 0.04,
    'seed':random_seed,
    'nthread':16
        }
    watchlist  = [(dtrain,'train')]
    model = xgb.train(params,dtrain,num_boost_round=num_boost_round,evals=watchlist)
    model.save_model('./model/xgb{0}.model'.format(iteration))
    
    #predict test set
    test_y = model.predict(dtest)
    test_result = pd.DataFrame(test_id,columns=["id"])
    test_result['score'] = test_y
    test_result.to_csv("./preds/xgb{0}.csv".format(iteration),index=None,encoding='utf-8')
    
    #get feature score
    feature_score = model.get_fscore()
    feature_score = sorted(feature_score.items(), key=lambda x:x[1],reverse=True)
    fs = []
    for (key,value) in feature_score:
        fs.append("{0},{1}\n".format(key,value))
    
    with open('./featurescore/feature_score_{0}.csv'.format(iteration),'w') as f:
        f.writelines("feature,score\n")
        f.writelines(fs)
    

In [5]:
%%time
random_seed = list(range(1000,2000,20))
gamma = [i/1000.0 for i in range(100,200,2)]
max_depth = [6,7,8]
lambd = list(range(200,400,2))
subsample = [i/1000.0 for i in range(600,700,2)]
colsample_bytree = [i/1000.0 for i in range(250,350,2)]
min_child_weight = [i/1000.0 for i in range(200,300,2)]
random.shuffle(random_seed)
random.shuffle(gamma)
random.shuffle(max_depth)
random.shuffle(lambd)
random.shuffle(subsample)
random.shuffle(colsample_bytree)
random.shuffle(min_child_weight)

with open('params.pkl','wb') as f:
    pickle.dump((random_seed,gamma,max_depth,lambd,subsample,colsample_bytree,min_child_weight),f)

#to reproduce my result, uncomment following lines
"""
with open('params_for_reproducing.pkl','rb') as f:
    random_seed,gamma,max_depth,lambd,subsample,colsample_bytree,min_child_weight = pickle.load(f)    
"""

for i in range(36):
    pipeline(dtrain,dvalid,valid_id,i,random_seed[i],gamma[i],max_depth[i%3],lambd[i],subsample[i],colsample_bytree[i],min_child_weight[i])

[0]	train-auc:0.74354
[1]	train-auc:0.786331
[2]	train-auc:0.793923
[3]	train-auc:0.800571
[4]	train-auc:0.805185
[5]	train-auc:0.808754
[6]	train-auc:0.810842
[7]	train-auc:0.811147
[8]	train-auc:0.813251
[9]	train-auc:0.814325
[10]	train-auc:0.814721
[11]	train-auc:0.816437
[12]	train-auc:0.817997
[13]	train-auc:0.818354
[14]	train-auc:0.819545
[15]	train-auc:0.821286
[16]	train-auc:0.822329
[17]	train-auc:0.822764
[18]	train-auc:0.823333
[19]	train-auc:0.824416
[20]	train-auc:0.824952
[21]	train-auc:0.825729
[22]	train-auc:0.825831
[23]	train-auc:0.826653
[24]	train-auc:0.827678
[25]	train-auc:0.828388
[26]	train-auc:0.828834
[27]	train-auc:0.829178
[28]	train-auc:0.83011
[29]	train-auc:0.830506
[30]	train-auc:0.830919
[31]	train-auc:0.831389
[32]	train-auc:0.832268
[33]	train-auc:0.832998
[34]	train-auc:0.833578
[35]	train-auc:0.834192
[36]	train-auc:0.834815
[37]	train-auc:0.835192
[38]	train-auc:0.835299
[39]	train-auc:0.836101
[40]	train-auc:0.836945
[41]	train-auc:0.836913
[42]

[334]	train-auc:0.942623
[335]	train-auc:0.942876
[336]	train-auc:0.942998
[337]	train-auc:0.943276
[338]	train-auc:0.94342
[339]	train-auc:0.943678
[340]	train-auc:0.943834
[341]	train-auc:0.944049
[342]	train-auc:0.944331
[343]	train-auc:0.944548
[344]	train-auc:0.944807
[345]	train-auc:0.945074
[346]	train-auc:0.945309
[347]	train-auc:0.945533
[348]	train-auc:0.945845
[349]	train-auc:0.946081
[350]	train-auc:0.946312
[351]	train-auc:0.946547
[352]	train-auc:0.946755
[353]	train-auc:0.946903
[354]	train-auc:0.947133
[355]	train-auc:0.947359
[356]	train-auc:0.9476
[357]	train-auc:0.947799
[358]	train-auc:0.947945
[359]	train-auc:0.948137
[360]	train-auc:0.948338
[361]	train-auc:0.948513
[362]	train-auc:0.948673
[363]	train-auc:0.948887
[364]	train-auc:0.949068
[365]	train-auc:0.949281
[366]	train-auc:0.94952
[367]	train-auc:0.949738
[368]	train-auc:0.949963
[369]	train-auc:0.950059
[370]	train-auc:0.950274
[371]	train-auc:0.95043
[372]	train-auc:0.95066
[373]	train-auc:0.950877
[374]	

[664]	train-auc:0.985917
[665]	train-auc:0.985959
[666]	train-auc:0.98602
[667]	train-auc:0.986096
[668]	train-auc:0.986151
[669]	train-auc:0.986226
[670]	train-auc:0.986257
[671]	train-auc:0.986331
[672]	train-auc:0.986401
[673]	train-auc:0.98647
[674]	train-auc:0.986546
[675]	train-auc:0.986599
[676]	train-auc:0.98666
[677]	train-auc:0.98674
[678]	train-auc:0.986758
[679]	train-auc:0.98682
[680]	train-auc:0.986884
[681]	train-auc:0.986948
[682]	train-auc:0.987031
[683]	train-auc:0.987106
[684]	train-auc:0.987152
[685]	train-auc:0.987179
[686]	train-auc:0.987225
[687]	train-auc:0.987291
[688]	train-auc:0.987326
[689]	train-auc:0.987392
[690]	train-auc:0.987457
[691]	train-auc:0.987475
[692]	train-auc:0.987539
[693]	train-auc:0.987577
[694]	train-auc:0.987651
[695]	train-auc:0.987702
[696]	train-auc:0.987752
[697]	train-auc:0.987813
[698]	train-auc:0.987877
[699]	train-auc:0.987944
[700]	train-auc:0.988015
[701]	train-auc:0.988055
[702]	train-auc:0.988105
[703]	train-auc:0.988177
[704]

[248]	train-auc:0.950903
[249]	train-auc:0.951173
[250]	train-auc:0.95153
[251]	train-auc:0.951869
[252]	train-auc:0.952126
[253]	train-auc:0.952376
[254]	train-auc:0.952652
[255]	train-auc:0.95294
[256]	train-auc:0.953144
[257]	train-auc:0.953384
[258]	train-auc:0.95352
[259]	train-auc:0.953761
[260]	train-auc:0.95398
[261]	train-auc:0.95418
[262]	train-auc:0.954442
[263]	train-auc:0.954552
[264]	train-auc:0.954849
[265]	train-auc:0.955096
[266]	train-auc:0.955292
[267]	train-auc:0.955403
[268]	train-auc:0.955619
[269]	train-auc:0.955813
[270]	train-auc:0.956091
[271]	train-auc:0.956292
[272]	train-auc:0.956493
[273]	train-auc:0.956766
[274]	train-auc:0.956907
[275]	train-auc:0.957111
[276]	train-auc:0.957365
[277]	train-auc:0.957554
[278]	train-auc:0.957848
[279]	train-auc:0.95803
[280]	train-auc:0.958284
[281]	train-auc:0.958475
[282]	train-auc:0.958693
[283]	train-auc:0.958915
[284]	train-auc:0.95917
[285]	train-auc:0.959384
[286]	train-auc:0.959547
[287]	train-auc:0.959698
[288]	t

[132]	train-auc:0.899532
[133]	train-auc:0.899727
[134]	train-auc:0.900232
[135]	train-auc:0.900876
[136]	train-auc:0.901373
[137]	train-auc:0.901782
[138]	train-auc:0.902089
[139]	train-auc:0.90261
[140]	train-auc:0.903059
[141]	train-auc:0.903641
[142]	train-auc:0.904047
[143]	train-auc:0.90453
[144]	train-auc:0.904974
[145]	train-auc:0.905497
[146]	train-auc:0.905925
[147]	train-auc:0.906404
[148]	train-auc:0.906908
[149]	train-auc:0.907274
[150]	train-auc:0.907578
[151]	train-auc:0.908021
[152]	train-auc:0.908565
[153]	train-auc:0.9089
[154]	train-auc:0.909284
[155]	train-auc:0.909641
[156]	train-auc:0.910075
[157]	train-auc:0.910438
[158]	train-auc:0.910739
[159]	train-auc:0.911271
[160]	train-auc:0.911697
[161]	train-auc:0.912142
[162]	train-auc:0.912382
[163]	train-auc:0.912868
[164]	train-auc:0.913189
[165]	train-auc:0.913728
[166]	train-auc:0.913981
[167]	train-auc:0.91439
[168]	train-auc:0.91459
[169]	train-auc:0.914918
[170]	train-auc:0.915345
[171]	train-auc:0.91571
[172]	t

...

[459]	train-auc:0.959804
[460]	train-auc:0.959979
[461]	train-auc:0.960184
[462]	train-auc:0.960328
[463]	train-auc:0.960455
[464]	train-auc:0.96053
[465]	train-auc:0.960677
[466]	train-auc:0.960766
[467]	train-auc:0.96094
[468]	train-auc:0.961111
[469]	train-auc:0.961246
[470]	train-auc:0.961376
[471]	train-auc:0.961506
[472]	train-auc:0.96168
[473]	train-auc:0.961856
[474]	train-auc:0.961914
[475]	train-auc:0.962061
[476]	train-auc:0.962185
[477]	train-auc:0.962362
[478]	train-auc:0.962469
[479]	train-auc:0.962638
[480]	train-auc:0.962771
[481]	train-auc:0.962923
[482]	train-auc:0.963059
[483]	train-auc:0.963225
[484]	train-auc:0.963386
[485]	train-auc:0.96352
[486]	train-auc:0.96364
[487]	train-auc:0.963787
[488]	train-auc:0.963948
[489]	train-auc:0.964093
[490]	train-auc:0.964264
[491]	train-auc:0.964434
[492]	train-auc:0.964577
[493]	train-auc:0.964746
[494]	train-auc:0.96487
[495]	train-auc:0.965039
[496]	train-auc:0.965147
[497]	train-auc:0.96527
[498]	train-auc:0.965404
[499]	t

[41]	train-auc:0.89071
[42]	train-auc:0.891366
[43]	train-auc:0.891871
[44]	train-auc:0.89253
[45]	train-auc:0.892945
[46]	train-auc:0.89354
[47]	train-auc:0.894657
[48]	train-auc:0.895288
[49]	train-auc:0.89554
[50]	train-auc:0.896208
[51]	train-auc:0.896694
[52]	train-auc:0.897242
[53]	train-auc:0.897811
[54]	train-auc:0.898528
[55]	train-auc:0.899017
[56]	train-auc:0.899541
[57]	train-auc:0.900322
[58]	train-auc:0.900645
[59]	train-auc:0.901241
[60]	train-auc:0.901472
[61]	train-auc:0.902432
[62]	train-auc:0.903122
[63]	train-auc:0.903762
[64]	train-auc:0.904758
[65]	train-auc:0.905272
[66]	train-auc:0.905776
[67]	train-auc:0.906535
[68]	train-auc:0.90737
[69]	train-auc:0.907795
[70]	train-auc:0.908397
[71]	train-auc:0.908962
[72]	train-auc:0.909343
[73]	train-auc:0.909973
[74]	train-auc:0.910475
[75]	train-auc:0.910791
[76]	train-auc:0.911373
[77]	train-auc:0.911843
[78]	train-auc:0.912335
[79]	train-auc:0.912925
[80]	train-auc:0.913728
[81]	train-auc:0.914321
[82]	train-auc:0.9146

[373]	train-auc:0.990052
[374]	train-auc:0.990157
[375]	train-auc:0.990196
[376]	train-auc:0.99027
[377]	train-auc:0.990322
[378]	train-auc:0.990392
[379]	train-auc:0.990455
[380]	train-auc:0.99054
[381]	train-auc:0.990609
[382]	train-auc:0.990708
[383]	train-auc:0.990761
[384]	train-auc:0.990834
[385]	train-auc:0.990906
[386]	train-auc:0.990961
[387]	train-auc:0.990978
[388]	train-auc:0.991035
[389]	train-auc:0.991095
[390]	train-auc:0.991173
[391]	train-auc:0.991235
[392]	train-auc:0.991302
[393]	train-auc:0.991356
[394]	train-auc:0.991418
[395]	train-auc:0.991476
[396]	train-auc:0.991559
[397]	train-auc:0.99161
[398]	train-auc:0.991683
[399]	train-auc:0.99175
[400]	train-auc:0.991794
[401]	train-auc:0.991862
[402]	train-auc:0.991924
[403]	train-auc:0.99197
[404]	train-auc:0.992031
[405]	train-auc:0.99209
[406]	train-auc:0.992162
[407]	train-auc:0.992223
[408]	train-auc:0.992283
[409]	train-auc:0.992324
[410]	train-auc:0.992366
[411]	train-auc:0.992437
[412]	train-auc:0.992474
[413]	

[257]	train-auc:0.936608
[258]	train-auc:0.936768
[259]	train-auc:0.936963
[260]	train-auc:0.937281
[261]	train-auc:0.937435
[262]	train-auc:0.937699
[263]	train-auc:0.93797
[264]	train-auc:0.938236
[265]	train-auc:0.938321
[266]	train-auc:0.9386
[267]	train-auc:0.938958
[268]	train-auc:0.93922
[269]	train-auc:0.93956
[270]	train-auc:0.939848
[271]	train-auc:0.940189
[272]	train-auc:0.940557
[273]	train-auc:0.940807
[274]	train-auc:0.941047
[275]	train-auc:0.941148
[276]	train-auc:0.941363
[277]	train-auc:0.94158
[278]	train-auc:0.941884
[279]	train-auc:0.942151
[280]	train-auc:0.942342
[281]	train-auc:0.942635
[282]	train-auc:0.942868
[283]	train-auc:0.943087
[284]	train-auc:0.94329
[285]	train-auc:0.943591
[286]	train-auc:0.94376
[287]	train-auc:0.943997
[288]	train-auc:0.944323
[289]	train-auc:0.944685
[290]	train-auc:0.944969
[291]	train-auc:0.945166
[292]	train-auc:0.945348
[293]	train-auc:0.945603
[294]	train-auc:0.945838
[295]	train-auc:0.946069
[296]	train-auc:0.946309
[297]	tr

[587]	train-auc:0.986156
[588]	train-auc:0.98623
[589]	train-auc:0.986311
[590]	train-auc:0.986374
[591]	train-auc:0.986456
[592]	train-auc:0.986533
[593]	train-auc:0.986612
[594]	train-auc:0.986621
[595]	train-auc:0.986711
[596]	train-auc:0.986781
[597]	train-auc:0.98686
[598]	train-auc:0.986926
[599]	train-auc:0.986978
CPU times: user 13h 18min 29s, sys: 29 s, total: 13h 18min 58s
Wall time: 52min 18s


In [ ]:
train_x.columns 

In [ ]:
valid.columns

In [ ]:
valid_d = pd.read_csv('../../preprocess_data/valid_float_rank_d.csv')

In [ ]:
valid_d.columns

In [ ]:
train_x_nd.head()